In [32]:
# suomeksi kirjoittavat, muut kielet

# import 

from collections import defaultdict, OrderedDict, Counter
from datetime import datetime

from dateutil.relativedelta import relativedelta
import glob
from itertools import product, combinations
import matplotlib.pyplot as plt
import numpy    as np
#import networkx as nx
from operator import itemgetter
import pandas   as pd
import rdflib as rdflib
import re
from scipy.signal import convolve2d
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer, minmax_scale
from SPARQLWrapper import SPARQLWrapper, JSON, POST, TURTLE
import sys
from rdflib.namespace import XSD, Namespace
from rdflib.term import URIRef

import IPython
import pprint
pp = pprint.PrettyPrinter(indent=4)



from constants import *
from helpers import *

In [33]:

from rdflib import Graph

from rdflib.namespace import RDF, SKOS
rel = rdflib.Namespace('http://ldf.fi/relse/')


# colors
import matplotlib.colors as mcolors
from matplotlib import cm
# set style
import seaborn as sns

#set_theme() 
sns.set_theme(style="white",palette="PuOr_r")

In [59]:
# Query
Q = """
PREFIX rel: <http://ldf.fi/relse/>

CONSTRUCT {


?teos a rel:novel ;
    skos:prefLabel ?title ;
    rel:langOrig ?kieliuri ;
    rel:hasAuthor ?author ;
    rel:pubYear ?vuosi_ .


?author a rel:author ;
	skos:prefLabel ?authorname ;
	foaf:gender ?gender ;
	rel:authorNationality ?kansallisuusuri .

?kieliuri a rel:lang ;
	skos:prefLabel ?kieli .

?kansallisuusuri a rel:nationality ;
	skos:prefLabel ?kansallisuus. 

}

WHERE {

  ?teos a kaunokki:romaani ; skos:prefLabel ?title .
  ?teos kaunokki:alkukieli ?kieliuri .
  FILTER (?kieliuri !=  <http://lexvo.org/id/iso639-3/fin>  )
  OPTIONAL {?kieliuri skos:prefLabel ?kieli . }
  ?teos kaunokki:manifests_in ?julkaisu .
  ?julkaisu kaunokki:kieli ?julk_kieli . # mikä vaan kieli
  ?julkaisu kaunokki:ilmestymisvuosi ?vuosi . ?vuosi skos:prefLabel ?vuosiluku .
  BIND(xsd:integer(?vuosiluku) AS ?vuosi_)
  ?teos kaunokki:tekija ?author.
  #?author skos:prefLabel ?authorname .
?author kaunokki:kansallisuus <http://www.yso.fi/onto/koko/p35259> . # suomal.
OPTIONAL  {?author foaf:gender ?gender }


} 
"""

# Query
Q2 = """
PREFIX rel: <http://ldf.fi/relse/>

CONSTRUCT {


?teos a rel:novel ;
    skos:prefLabel ?title ;
    rel:langOrig ?kieliuri ;
    rel:hasAuthor ?author ;
    rel:pubYear ?vuosi_ .


?author a rel:author ;
	skos:prefLabel ?authorname ;
	foaf:gender ?gender ;
	rel:authorNationality ?kansallisuusuri .

?kieliuri a rel:lang ;
	skos:prefLabel ?kieli .

?kansallisuusuri a rel:nationality ;
	skos:prefLabel ?kansallisuus. 

}

WHERE {

  ?teos a kaunokki:romaani ; skos:prefLabel ?title .
  ?teos kaunokki:alkukieli ?kieliuri .
  #FILTER (?kieliuri !=  <http://lexvo.org/id/iso639-3/fin>  ) # ei suomi
  #OPTIONAL {?kieliuri skos:prefLabel ?kieli . }
  ?teos kaunokki:manifests_in ?julkaisu .
  ?julkaisu kaunokki:kieli <http://lexvo.org/id/iso639-3/fin> . # julkaistu suomeksi
  ?julkaisu kaunokki:ilmestymisvuosi ?vuosi . ?vuosi skos:prefLabel ?vuosiluku .
  BIND(xsd:integer(?vuosiluku) AS ?vuosi_)
  ?teos kaunokki:tekija ?author.
  #?author skos:prefLabel ?authorname .
?author kaunokki:kansallisuus <http://www.yso.fi/onto/koko/p35259> . # suomal.
OPTIONAL  {?author foaf:gender ?gender }


} 
"""

In [60]:
print('\n\n*** constructing the graph')
sparql = SPARQLWrapper("http://ldf.fi/booksampo-2022/sparql")

sparql.setQuery(PREFIXES + Q)

sparql.addCustomHttpHeader(*list(AUTHORIZATION_HEADER.items())[0])

sparql.setReturnFormat(TURTLE)
results = sparql.query().convert()



*** constructing the graph


In [61]:
f = Graph()
#g.parse(data=results, format="turtle") # "mygraph.ttl"
f.parse(data=results, format="turtle")

<Graph identifier=N2aadd622e8b34afab073db0155bfa698 (<class 'rdflib.graph.Graph'>)>

In [62]:
print('\n\n*** constructing the graph')
sparql = SPARQLWrapper("http://ldf.fi/booksampo-2022/sparql")

sparql.setQuery(PREFIXES + Q2)

sparql.addCustomHttpHeader(*list(AUTHORIZATION_HEADER.items())[0])

sparql.setReturnFormat(TURTLE)
results = sparql.query().convert()



*** constructing the graph


In [63]:
F = Graph()
#g.parse(data=results, format="turtle") # "mygraph.ttl"
F.parse(data=results, format="turtle")

<Graph identifier=N6e829f47edc146cebcde705fff64e7f1 (<class 'rdflib.graph.Graph'>)>

In [64]:
# suomen kansalaiset, ei-suomenkiel. teokset

Q ="""
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX kaunokki: <http://www.yso.fi/onto/kaunokki#> 
PREFIX rel: <http://ldf.fi/relse/> 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT DISTINCT ?teos ?kieli ?year  ?author ?kieli
{
  ?teos a rel:novel .
  
  ?teos rel:hasAuthor ?author .
  #?author rel:authorNationality <http://www.yso.fi/onto/koko/p35259> .
  #?kansallisuusuri skos:prefLabel ?nationality_. FILTER(LANG(?nationality_)="fi")
  
 ?teos rel:langOrig ?kieli .
  ?teos rel:pubYear ?year .
 FILTER(?year > 1970)
 FILTER(?year < 2021)
  
} #GROUP BY  ?year ?kieli

"""

qres = f.query(Q)
resdict=[{"year": int(row.asdict()['year'].toPython()),"teos":str(row.asdict()['teos'].toPython()), "author":str(row.asdict()['author'].toPython()), "language": str(row.asdict()['kieli'].toPython())}  for row in qres]


In [65]:
# suomen kansalaiset, muunkieliset teokset

Q ="""
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX kaunokki: <http://www.yso.fi/onto/kaunokki#> 
PREFIX rel: <http://ldf.fi/relse/> 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT DISTINCT ?teos ?kieli ?year  ?author ?kieli
{
  ?teos a rel:novel .
  
  ?teos rel:hasAuthor ?author .
  #?author rel:authorNationality <http://www.yso.fi/onto/koko/p35259> .
  #?kansallisuusuri skos:prefLabel ?nationality_. FILTER(LANG(?nationality_)="fi")
  
 ?teos rel:langOrig ?kieli .
  ?teos rel:pubYear ?year .
 FILTER(?year > 1970)
 FILTER(?year < 2021)
  
} #GROUP BY  ?year ?kieli

"""

qres = F.query(Q)
resdictF=[{"year": int(row.asdict()['year'].toPython()),"teos":str(row.asdict()['teos'].toPython()), "author":str(row.asdict()['author'].toPython()), "language": str(row.asdict()['kieli'].toPython())}  for row in qres]


In [66]:
# suomalaisten muunkieliset
finlangdf=pd.DataFrame(resdict).sort_values(by="year")
finlangdf.head()

,year,teos,author,language
92,1971,http://seco.tkk.fi/saha3/u35cdd023-8b09-4d54-8...,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/swe
874,1971,http://seco.tkk.fi/saha3/ua03e39a7-b0a7-46b0-8...,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/swe
67,1971,http://www.yso.fi/onto/kaunokki#ateos_3660,http://www.yso.fi/onto/kaunokki#person_1231760...,http://lexvo.org/id/iso639-3/swe
444,1971,http://seco.tkk.fi/saha3/uce78311d-2324-43e7-9...,http://www.yso.fi/onto/kaunokki#person_1232726...,http://lexvo.org/id/iso639-3/swe
60,1971,http://seco.tkk.fi/saha3/ua25d6048-3595-4ef4-a...,http://seco.tkk.fi/saha3/u40ef2414-b89e-4656-a...,http://lexvo.org/id/iso639-3/swe


In [67]:
#kaikki suomalaisten teokset
finlangFI=pd.DataFrame(resdictF).sort_values(by="year")
finlangFI.head()

,year,teos,author,language
12638,1971,http://www.yso.fi/onto/kaunokki#ateos_33404,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/fin
15379,1971,http://www.yso.fi/onto/kaunokki#ateos_30894,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/fin
7736,1971,http://www.yso.fi/onto/kaunokki#ateos_49500,http://www.yso.fi/onto/kaunokki#person_1231760...,http://lexvo.org/id/iso639-3/fin
7778,1971,http://seco.tkk.fi/saha3/u50c30d8c-ff58-4b13-8...,http://seco.tkk.fi/saha3/u943c6db5-c41a-4444-b...,http://lexvo.org/id/iso639-3/fin
7794,1971,http://www.yso.fi/onto/kaunokki#ateos_48816,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/fin


In [68]:
# add period function
def add_period(df_fo, columnname="year"):
    df_fo['period'] = pd.cut(df_fo[columnname], bins=bins, include_lowest=True, precision=0)
    df_fo['period'] = df_fo['period'].astype("str")

    df_fo["period"]=df_fo["period"].apply(lambda x: int(x.split(",")[0].replace("[","")))
    return df_fo

# decade function
def add_decade(df_fi, columname="year"):
    df_fi["decade"]=(df_fi["year"]-1)- ((df_fi["year"]-1)%10)
    
    return df_fi

In [69]:
finlangdf=add_decade(finlangdf)


In [70]:
finlangdf.head()

,year,teos,author,language,decade
92,1971,http://seco.tkk.fi/saha3/u35cdd023-8b09-4d54-8...,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/swe,1970
874,1971,http://seco.tkk.fi/saha3/ua03e39a7-b0a7-46b0-8...,http://www.yso.fi/onto/kaunokki#person_1231759...,http://lexvo.org/id/iso639-3/swe,1970
67,1971,http://www.yso.fi/onto/kaunokki#ateos_3660,http://www.yso.fi/onto/kaunokki#person_1231760...,http://lexvo.org/id/iso639-3/swe,1970
444,1971,http://seco.tkk.fi/saha3/uce78311d-2324-43e7-9...,http://www.yso.fi/onto/kaunokki#person_1232726...,http://lexvo.org/id/iso639-3/swe,1970
60,1971,http://seco.tkk.fi/saha3/ua25d6048-3595-4ef4-a...,http://seco.tkk.fi/saha3/u40ef2414-b89e-4656-a...,http://lexvo.org/id/iso639-3/swe,1970


In [71]:
finlangdf=finlangdf.replace({"http://www.lingvoj.org/lang/smi":"http://lexvo.org/id/iso639-5/smi"})

In [72]:
swe=finlangdf[finlangdf["language"].str.contains("/swe")].groupby("decade")["teos"].nunique()
engnovels=finlangdf[finlangdf["language"].str.contains("/eng")].groupby("decade")["teos"].nunique()

notswe=finlangdf[~finlangdf["language"].str.contains("/swe")].groupby("decade")["teos"].nunique()
notswe_au=finlangdf[~finlangdf["language"].str.contains("/swe")].groupby("decade")["author"].nunique()

allnovels=finlangdf.groupby(["decade"]).teos.nunique()

allauthors=finlangdf.groupby(["decade"]).author.nunique()

rel_notswe=notswe/allnovels
rel_eng=engnovels/allnovels
rel_swe=swe/allnovels

In [78]:
# kielet
nlangfi=finlangdf.groupby(["decade"]).language.nunique()
print(nlangfi)


decade
1970     1
1980     5
1990     6
2000     7
2010    14
Name: language, dtype: int64


In [82]:
# käännetyt teokset

finlangFI=add_decade(finlangFI)

nnovelfi=finlangFI[finlangFI["language"]!="http://lexvo.org/id/iso639-3/fin"].groupby(["decade"]).teos.nunique()
print(nnovelfi)
# add column for translations into Finnish?
kaannokset=[ 73  ,80 , 61, 132, 172]
kaann=pd.Series(kaannokset)
kaann

decade
1970     73
1980     80
1990     61
2000    132
2010    172
Name: teos, dtype: int64


0     73
1     80
2     61
3    132
4    172
dtype: int64

In [79]:
decdf=pd.concat([allnovels, nlangfi, rel_swe,rel_eng],axis=1)
decdf.reset_index(inplace=True)
decdf.columns=["vk","lkm","kieli","swe","rel_eng"]
decdf["suomennetut"]=kaann
decdf["suomennetut"]=decdf["suomennetut"]/decdf["lkm"]
print(round(decdf.fillna(0),2).to_latex(index=False))

\begin{tabular}{rrrrrr}
\toprule
  vk &  lkm &  kieli &  swe &  rel\_eng &  suomennetut \\
\midrule
1970 &  222 &      1 & 1.00 &     0.00 &         0.33 \\
1980 &  242 &      5 & 0.97 &     0.00 &         0.33 \\
1990 &  260 &      6 & 0.97 &     0.00 &         0.23 \\
2000 &  397 &      7 & 0.95 &     0.02 &         0.33 \\
2010 &  526 &     14 & 0.91 &     0.04 &         0.33 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_9418/3922002862.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(round(decdf.fillna(0),2).to_latex(index=False))


In [49]:

finlangdf.groupby(["decade"])[["teos","author"]].nunique()

,teos,author
decade,,
1970,222,99
1980,242,124
1990,260,146
2000,397,212
2010,526,277


In [50]:
print(finlangdf[finlangdf["year"]>2000]["teos"].nunique())
print(finlangdf[(finlangdf["year"]>2000)&(finlangdf["language"].str.contains("/swe"))]["teos"].nunique())

print(finlangdf[finlangdf["year"]>2000]["author"].nunique())
print(finlangdf[(finlangdf["year"]>2000)&(~finlangdf["language"].str.contains("/swe"))]["author"].nunique())

907
844
402
37


In [51]:
print(finlangdf[finlangdf["year"]>2000]["teos"].nunique())
print(finlangdf[(finlangdf["year"]>2000)&(finlangdf["language"].str.contains("/eng"))]["teos"].nunique())

print(finlangdf[finlangdf["year"]>2000]["author"].nunique())
print(finlangdf[(finlangdf["year"]>2000)&(finlangdf["language"].str.contains("/eng"))]["author"].nunique())

907
28
402
21


In [52]:
print(finlangFI[finlangFI["year"]>2000]["teos"].nunique())
#print(finlangdf[(finlangdf["year"]>2000)&(finlangdf["language"].str.contains("/eng"))]["teos"].nunique())

print(finlangFI[finlangFI["year"]>2000]["author"].nunique())

11543
4431


In [83]:
notswecount=finlangdf[(finlangdf["year"]>2010)&(~finlangdf["language"].str.contains("/swe"))]["teos"].nunique()
allcount=finlangFI[finlangFI["year"]>2010]["teos"].nunique()
#907/11535 , 402/4430
print(notswecount/allcount)

0.0070921985815602835


In [84]:
notswecount=finlangdf[(finlangdf["year"]>1990)&(~finlangdf["language"].str.contains("/swe"))&(finlangdf["year"]<2001)]["teos"].nunique()
allcount=finlangFI[(finlangFI["year"]>1990)&(finlangdf["year"]<2001)]["teos"].nunique()
#907/11535 , 402/4430
print(notswecount/allcount)

0.011217948717948718


/tmp/ipykernel_9418/1023933337.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  allcount=finlangFI[(finlangFI["year"]>1990)&(finlangdf["year"]<2001)]["teos"].nunique()
